# Ridge und Lasso

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import Gauss
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

plt.rcParams["figure.figsize"] = (10,8)


## Overfitting

Die Einführung von Basisfunktionen in unsere lineare Regression macht das Modell
sehr viel flexibler, kann aber auch sehr schnell zu einer Überanpassung (Overfitting) führen.

Wenn wir z.B. zu viele Gauß'sche Basisfunktionen wählen, erhalten wir am Ende
Ergebnisse, die nicht so gut aussehen:

In [ ]:
rng = np.random.default_rng(1)
x = 10 * rng.random(50)
y = np.sin(x) + 0.1 * rng.standard_normal(50)

model = make_pipeline(Gauss.GaussianFeatures(30),
                      LinearRegression())
model.fit(x[:, np.newaxis], y)

plt.scatter(x, y)

xfit = np.linspace(0, 10, 1000)
plt.plot(xfit, model.predict(xfit[:, np.newaxis]))

plt.xlim(0, 10)
plt.ylim(-1.5, 1.5);

Da die Daten auf die 30-dimensionale Basis projiziert werden, ist das Modell viel
zu flexibel und nimmt Extremwerte an, um sich möglichst genau an die Daten
anzupassen. Das Problem dabei ist, dass Vorhersagen für neue Werte dann nichts taugen, denn die Kurve hat
sich auf die vorhandenen Werte spezialisiert und lässt sich nicht verallgemeinern.

Wir können den Grund dafür erkennen, wenn wir die Koeffizienten der Gauß'schen Basen
in Relation zu $x$ darstellen:

In [ ]:
def basis_plot(model, title=None):
    fig, ax = plt.subplots(2, sharex=True)
    model.fit(x[:, np.newaxis], y)
    ax[0].scatter(x, y)
    ax[0].plot(xfit, model.predict(xfit[:, np.newaxis]))
    ax[0].set(xlabel='x', ylabel='y', ylim=(-1.5, 1.5))

    if title:
        ax[0].set_title(title)

    ax[1].plot(model.steps[0][1].centers_,
               model.steps[1][1].coef_)
    ax[1].set(xlabel='basis location',
              ylabel='coefficient',
              xlim=(0, 10))

model = make_pipeline(Gauss.GaussianFeatures(30), LinearRegression())
basis_plot(model)

Das untere Feld dieser Abbildung zeigt die Amplitude der Basisfunktion an jedem
Ort. Dies ist ein typisches Overfitting-Verhalten, wenn sich Basisfunktionen
überlappen: die Koeffizienten benachbarter Basisfunktionen explodieren und heben
sich gegenseitig auf.

## Regularisierung

Wir wissen, dass ein solches Verhalten problematisch ist, und es wäre schön, wenn
wir solche Spitzen im Modell explizit begrenzen könnten, indem wir große Werte der
Modellparameter bestrafen. Eine solche Bestrafung ist als **Regularisierung**
bekannt und kommt in verschiedenen Formen vor.

## Ridge-Regression ($L_2$ Regularisierung)

Die vielleicht häufigste Form der Regularisierung ist bekannt als Ridge-Regression
oder $L_2$-Regularisierung. Dabei wird die Summe der Quadrate (2-Norm) der
Modellkoeffizienten $\theta_i$ bestraft; in diesem Fall würde die Strafe für die Modellanpassung

$P=\alpha\sum_{n=1}^{N}\theta_n^2$

wobei $\alpha$ ein freier Parameter ist, der die Stärke der Strafe steuert. Diese
Art von Bestrafungsmodell ist in Scikit-Learn mit dem Ridge-Schätzer eingebaut:

In [ ]:
from sklearn.linear_model import Ridge, LinearRegression
model = make_pipeline(Gauss.GaussianFeatures(30), Ridge(alpha=0.1))
basis_plot(model, title='Ridge Regression')

Der Parameter $\alpha$ ist im Wesentlichen ein Regler, der die Komplexität des
resultierenden Modells steuert. Im Limit $\alpha \to 0$ erhalten wir das lineare
Standard-Regressionsergebnis zurück; im Limit $\alpha \to \infty$ werden alle
Modellantworten unterdrückt.

Ein Vorteil der Ridge-Regression besteht insbesondere darin, dass sie sehr
effizient berechnet werden kann - mit kaum mehr Rechenaufwand als das
ursprüngliche lineare Regressionsmodell.

## Lasso-Regression ($L_1$-Regulierung)

Eine andere sehr verbreitete Art der Regularisierung ist als "Lasso" bekannt und
beinhaltet die Bestrafung der Summe der absoluten Werte (1-Normen) der
Regressionskoeffizienten:

$P=\alpha\sum_{n=1}^N|\theta_n|$

Obwohl dies konzeptionell der Ridge-Regression sehr ähnlich ist, können die
Ergebnisse überraschend unterschiedlich ausfallen:

Beispielsweise neigt die Lasso-Regression aus geometrischen Gründen dazu, wenn
möglich "sparse" Modelle zu bevorzugen, d.h. sie setzt die Modellkoeffizienten
bevorzugt genau auf Null - eine Art der Feature-Selektion.

Wir können dieses Verhalten beobachten, wenn wie die Koeffizienten der Lasso-Regression abtragen:

In [ ]:
from sklearn.linear_model import Lasso
model = make_pipeline(Gauss.GaussianFeatures(30), Lasso(alpha=0.001))
basis_plot(model, title='Lasso Regression')

Bei der Lasso-Regression ist die Mehrzahl der Koeffizienten genau null, wobei das 
funktionale Verhalten durch eine kleine Untermenge der verfügbaren Basisfunktionen 
modelliert wird. 

Wie bei der Ridge-Regularisierung bestimmt der Hyperparameter $\alpha$ die Stärke der 
Strafe und sollte z.B. durch Kreuzvalidierung bestimmt werden.

## Plot von $L_1$ und $L_2$

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(15,7))

x = np.linspace(-1,1,100)
y = np.asarray([np.abs(x), x**2])
for i in [0,1]:
    axs[i].plot(x, y[i])
    axs[i].axvline(0, c='grey')
    axs[i].axhline(0, c='grey')